# Drop all columns with no values

In [ ]:
# Drop all columns with no values
df.dropna(axis=1,how='all',inplace=True)

In [ ]:
def gspc_formatting(df):
    #column index, column name, values, allow duplicates (optional)
    df.insert(0, 'MFD', '')
    df.insert(0, 'ID', range(1, 1 + len(df)))
    df.insert(0, 'Source', 'Source Name')

# Reasons to exclude

- Duplicate
- Invalid Award Amount
- Less than 1000
- No Vendor
- Null Award Amount
- Exclusion Category 
- Award Status

# Checking if character string substring is in cell

In [ ]:
def is_mbe(string):
    if('MBE' in string):
        return True
    else:
        return False
    
dbe['Is MBE'] = False
dbe['Is MBE'] = dbe['Certifications'].apply(is_mbe)

# Disable Autosave

In [ ]:
%autosave 0

# Check if string contains number letter

In [ ]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

# Python Index based loops

In [ ]:
fruits = ['banana', 'apple',  'mango']

for index in range(len(fruits)):
    print('Current fruit :', fruits[index])

# Check if item is in an array / list / dictionary

In [ ]:
#This works for any collection, not just for lists. For dictionaries, it checks whether the given key is 
#present in the dictionary.

if item in my_list:
    # whatever

# Firm Name Cleaning

In [ ]:
# https://medium.com/@isma3il/supplier-names-normalization-part1-66c91bb29fc3

# Libraries
import os, re
import pandas as pd
import np
from cleanco import cleanco
# Import supplier names to dataframe
df = pd.read_excel('EMEA-spend.xlsx', names=['Supplier_Name'], usecols=[5])
# ----------------------------------------
# Convert to uppercase
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.str.upper()
# Remove commas
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.str.replace(',', '')
# Remove hyphens
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.str.replace(' - ', ' ')
# Remove text between parenthesis 
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.str.replace(r"\(.*\)","")
#
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.str.replace(' AND ', ' & ')
# Remove spaces in the begining/end
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.str.strip()
# Encode
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.str.encode('utf-8')
# Remove business entities extensions (1)
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.apply(lambda x: cleanco(x).clean_name() if type(x)==str else x)
# Remove dots
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.str.replace('.', '')
# Remove business entities extensions (2) - after removing the dots
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.apply(lambda x: cleanco(x).clean_name() if type(x)==str else x)
# Specific Polish to companies
df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.str.replace('SP ZOO', '')
# Remove European country names from supplier names
countries = ['ALBANIA', 'ANDORRA', 'AUSTRIA', 'AZERBAIJAN', 'BELARUS', 'BELGIUM', 'BOSNIA AND HERZEGOVINA', 'BULGARIA', 'CROATIA', 'CYPRUS', 'CZECH REPUBLIC', 'DENMARK', 'ESTONIA', 'FAROE ISLANDS', 'FINLAND', 'FRANCE', 'BRITTANY', 'GERMANY', 'DEUTSCHLAND', 'GREECE', 'GUERNSEY (CHANNEL ISLANDS)', 'HUNGARY', 'ICELAND', 'IRELAND', 'ISLE OF MAN', 'ITALY', 'JERSEY (CHANNEL ISLANDS)', 'LATVIA', 'LIECHTENSTEIN', 'LITHUANIA', 'LUXEMBOURG', 'MACEDONIA', 'MALTA', 'MOLDOVA', 'MONACO', 'MONTENEGRO', 'NETHERLANDS', 'NEDERLAND', 'HOLLAND', 'NORWAY', 'POLAND', 'POLSKA', 'PORTUGAL', 'ROMANIA', 'RUSSIA', 'SAN MARINO', 'SERBIA', 'SLOVAKIA', 'SLOVENIA', 'SPAIN', 'SWEDEN', 'SWITZERLAND', 'TURKEY', 'UNITED KINGDOM', 'UK', 'SCOTLAND', 'WALES', 'CORNWALL', 'NORTHERN IRELAND', 'UKRAINE', 'VATICAN CITY']
for country in countries:
    df.Supplier_Name_Normalized = df.Supplier_Name_Normalized.apply(lambda x: x.replace(country, '') if (type(x)==str and x.endswith(country)) else x)
# Count unique values
print 'Supplier names:', df.Supplier_Name.nunique()
print 'Normalized names:', df.Supplier_Name_Normalized.nunique()

# Converting NAICS to NIGP

In [ ]:
crosswalk = pd.read_excel('../NAICS-CROSSWALK.xlsx')

crosswalk['NAICS'] = crosswalk['NAICS'].astype(str)
crosswalk['NIGP_CODE'] = crosswalk['NIGP_CODE'].astype(str)
crosswalk['New NAICS'] = crosswalk['NAICS']
crosswalk = crosswalk[['New NAICS','NIGP_CODE','NIGP_Description']]

blank_work_with_converted_nigp = blank_work_with_naics.merge(crosswalk,on='New NAICS',how='left')

#Reducing length of NIGP to its 3 and 5 digit version so it can be merged on our file of specific codes to care about
successful_conversions['NIGP 3'] = successful_conversions['NIGP_CODE'].apply(lambda x: x[:3])
successful_conversions['NIGP 5'] = successful_conversions['NIGP_CODE'].apply(lambda x: x[:5])

three = pd.read_csv('../three.csv')
five = pd.read_csv('../five.csv')

three.rename(columns={'NIGP':'NIGP 3','Work Category':'Merged Work 3'},inplace=True)
five.rename(columns={'NIGP':'NIGP 5','Work Category':'Merged Work 5'},inplace=True)

three['NIGP 3']=three['NIGP 3'].astype(str)
five['NIGP 5']=five['NIGP 5'].astype(str)

successful_conversions = successful_conversions.merge(three,on='NIGP 3',how='left')
successful_conversions = successful_conversions.merge(five,on='NIGP 5',how='left')

# Import multiple files directory

In [ ]:
#https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
    
import pandas as pd
import glob

path = r'C:\Users\Griffin Strong\OneDrive\Omar\My Notebooks\NCDOT\data\190416\FY 2014 Disparity Study Files\Invoice Data\Invoice 1' # use your path
all_files = glob.glob(path + "/*.xlsx")

li = []

for filename in all_files:
    df = pd.read_excel(filename, index_col=None, header=0)
    li.append(df)

invoice_1 = pd.concat(li, axis=0, ignore_index=True)

# Exporting Excel Files

In [ ]:
writer = pd.ExcelWriter('../data/interim/revised_po_with_correct_amounts.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1',index=False)
writer.save()

# Data frame value counter frequency counter

In [ ]:
def valueCounter(df,path):
    f= open(path,"w+")
    for i in range(df.shape[1]):
        colname = df.columns[i]
        f.write('Column Name: '+colname+'\n')
        f.write('Number of Values: ' + str(len(df[colname]))+'\n')
        f.write('Number of Unique Values: ' + str(len(df[colname].unique()))+'\n')
        f.write('-----------------------------------------------------------'+'\n')
        f.write(str(df.iloc[:,i].value_counts(dropna=False))+'\n')
        f.write('\n\n')
        
    f.close()

# Exclusion Algorithm

In [ ]:
#County Names, State Names, City Names, Federal Agencies
exclusion_list = [
'Chattanooga',
'Tennessee',
'TN',
'City',
'County',
'Sheriff', 
'Department',
'Fire',
'Country',
'States',
'State',
'United',
'U.S.',
'Federal',
'Council',
'Government',
'Govt.',
'Church',
'Ministries',
'Ministry',
'Theaters',
'Theater',
'Clubs',
'Foundations',
'Foundation',
'Publishing',
'Institute',
'Association',
'League',
'Libraries',
'Library',
'Scouts',
'Scout',
'School',
'Daycare',
'Learning',
'Chapter',
'Commissions',
'Commission',
'YMCA',
'YWCA',
'College',
'University',
'Society',
'Chamber',
'Hospitals',
'Hospital',
'Bureau',
'Department',
'Agency',
'National',
'Goodwill',
'Salvation',
'Unions',
'Union',
'Catholic',
'Baptist',
'Methodist',
'Muslim',
'Jewish',
'Presbyterian',
'Episcopal',
'Junior',
'Circuit',
'Court',
'Alliance',
'District',
'A.S.S.N',
'Habitat',
'Humanity',
'Non-profit',
'Organization',
'Authority',
'Center',
'Development',
'Campaign',
'Conference',
'Board',
'Division',
'Awareness',
'Christian',
'Museum',
'Charity',
'Health Care',
'Health',
'Dept',
'Coalition', 
'Collaborative',
'Books',
'Magazine',
'Grants',
'Grant',
'Lease-property',
'Wages',
'Land',
'Reimbursements',
'Reimbursement',
'Expenses',
'Expense',
'Utilities',
'Utility',
'Trust',
'Park',
'Natl',
'Committee',
'Sports',
'Sport',
'Habitat',
'Centers',
'Treasury',
'Dep',
'United',
'Govrnt',
'Gov',
'Govt',
'YMCA',
'Community',
'Assn',
'Univ',
'Municipal',
'Police',
'Assoc',
'Cross',
'Public',
'Safety',
'Agncy',
'Enforcement',
'Bank',
'Children',
'Partnership',
'Nursery',
'Depart',
'Postal'
]

def mfd_exclusions(df,name_column):
    df['Potentially Exclude'] = False
    for index, row in df.iterrows(): 
        array = df.iloc[index][name_column].title().split()
        #print(array)
        for i in range(len(array)):
            #print(i)
            if array[i] in exclusion_list:
                #print(i)
                df.at[index,'Potentially Exclude'] = True
                
mfd_exclusions(master,'Business Name')

# String Extraction

In [ ]:
def extract_city(string):
    array = str(string).split(',')
    return array[0]

tnucp['Real City'] = tnucp['City'].apply(extract_city)

# Exporting CSV Files

In [ ]:
mvf.to_csv('path',index=False)

# Reduce columns, Delete columns

In [ ]:
fy13 = fy13[['col1','col2']]

In [ ]:
df.drop(['B', 'C'], axis=1) #Deletes column B and C

# Find intersection of two dataframe's column name

In [ ]:
set(po_item.columns).intersection(set(po_header.columns))

# Resolve Multiple Addresses

In [ ]:
hub.loc[hub['Address 1'].notnull()&hub['Address 2'].isnull(),'Address'] = hub['Address 1']
hub.loc[hub['Address 1'].notnull()&hub['Address 2'].notnull(),'Address'] = hub['Address 1'].astype(str) + ' ' + hub['Address 2'].astype(str)

# Print list of columns fill when align aligning compare

In [ ]:
np.setdiff1d(['MFD','Source', 
       'Active Vendor Flag','Vendor Number',  
       'Business Name', 'Smoothed Name', 'Owner', 'Phone', 'Email', 'Address',
       'City', 'County', 'State', 'Zip', 'DBE Category', 'Work Description',
       'NIGP 5 Vendor', 'NIGP 3 Vendor', 'All Vendor NIGP', 'Ethnicity',
        'Vendor Work Categorization Type',
       'Potential Minority Identifier', 'Relevant Market Region',
       'Potentially Exclude', 'Actually Exclude'],hub.columns)

for i in array:
    print('df[\''+i+'\'] = np.nan')

# Mark Duplicates

In [ ]:
def mark_duplicates(df,column_list):
    df['Duplicate']=df.duplicated(subset=column_list)
    dup_criteria = ''
    for string in column_list:
        dup_criteria = dup_criteria+string+', '
    df.loc[df['Duplicate']==True,'MFD'] = 'Duplicate by ' + dup_criteria
    df.drop(['Duplicate'],axis=1,inplace=True)
    return df

master=mark_duplicates(df,['Business Name','Email','County'])

In [ ]:
def mark_duplicates(df,column_list):
    df['Duplicate']=df.duplicated(subset=column_list)
    df.loc[df['Duplicate']==True,'MFD'] = 'Duplicate by PO_NBR'
    exclusions = df[df['MFD'].notnull()].copy()
    df = df[df['MFD'].isnull()].copy()
    
    return df, exclusions

priority_frame,exclusions_1=mark_duplicates(priority_frame,['PO_NBR'])

# Getting the frequency count number of times a value appears in a data frame

In [ ]:
temp = DF_GOES_HERE.drop_duplicates(subset=['Cleaned Name','Hub Certification'])
def createDupNameFrame(df,name_column):
    name_counts = df[name_column].value_counts().rename('name_counts')

    dup_names = df.merge(name_counts.to_frame(),
                                    left_on=name_column,
                                    right_index=True)

    dup_names = dup_names[dup_names['name_counts']>1]
    return dup_names

dup_names = createDupNameFrame(temp,'NAME COLUMN GOES HERE')
pd.set_option('display.max_rows',None)
dup_names

In [ ]:
def createDupNameFrame(df,name_column):
    name_counts = df[name_column].value_counts().rename('Number of Instances')

    dup_names = df.merge(name_counts.to_frame(),
                                    left_on=name_column,
                                    right_index=True)
    dup_names['Number of Instances Key'] = name_column
#     dup_names = dup_names[dup_names['name_counts']>1]
    return dup_names

DF_GOES_HERE = createDupNameFrame(DF_GOES_HERE,'NAME COLUMN GOES HERE')

In [ ]:
name_counts = mwbe_no_null_eth['Po Supplier Name'].value_counts().rename('name_counts')

ethnicity_conflicts = mwbe_no_null_eth.merge(name_counts.to_frame(),
                                left_on='Po Supplier Name',
                                right_index=True)

ethnicity_conflicts = ethnicity_conflicts[ethnicity_conflicts['name_counts']>1]

# Regular Expression Req Number regex

In [ ]:
import re

def clean_req(req):
    x = re.findall('([\d]{5})', req)
    if not x:
        return np.nan
    else:
        string = ';'.join(x)
        return(string)

master['Cleaned Req Number 2'] = master['Cleaned Req Number'].astype(str).apply(clean_req)

# Return a two 2 values dataframes from a function

In [ ]:
merge_results,po_iteration=ethnicity_pull(po,mwbe,'Po Supplier Name','Cleaned Name',False)

# Set column as string

In [ ]:
master['Email'] = master['Email'].str
df['Item Category'] =df['Item Category'].astype(str)

# Remove replace substring from string

In [ ]:
y = x.replace('.good','')

# Create a columns

In [ ]:
df.insert(0,'MFD','')
df['NIGP 5'] = ''
df['NIGP 3'] = ''

# Insert Column

In [ ]:
chat.insert(0,'Source','Source File Name')

# Slice shorten strings

In [ ]:
df['NIGP 5'] = df['NIGP'].str.slice(0,5)

# VLOOKUP, merge

In [ ]:
results_fy13 = fy13.merge(three,on='NIGP 3',how='left')
# Note, three is the table we are looking up values on, fy13 is the main
# table, and NIGP 3 is the shared column nam

# Remove dash, commas, character from string

In [ ]:
df['NIGP 7'] = df['NIGP 7'].str.replace("-","")

# Display entire cell content full non-truncated

In [ ]:
pd.set_option('display.max_colwidth', -1)

# Concatenating data frame

In [ ]:
master = pd.concat([df1,df2])

# Remove Scientific, Globally setting the float length

In [ ]:
pd.options.display.float_format = '{:.5f}'.format

# Changing default number of rows all columns displayed show all

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Extracting values from comma separated cells new line

In [ ]:
tnucp.rename(columns={0:'Col0',1:'Col1',2:'Col2',3:'Col3',4:'Col4',5:'Col5'},inplace=True)

def clean_firm_name(df):
    for index, row in df.iterrows():
        array = str(df.iloc[index]['Company Information ']).split('\n')
        for i in range(len(array)):
            df.at[index,i] = array[i]
            
    #New addition, may break it, need to expand this function to be able to take this    
    tnucp.rename(columns={0:'Col0',1:'Col1',2:'Col2',3:'Col3',4:'Col4',5:'Col5'},inplace=True)


# Find duplicated rows 

In [ ]:
#Note you want to check that there are now Emails/Phones with blanks because those can be considered duplicates
#master_women[master_women['Email'].apply(len) < 4]
#master_women[master_women['Phone'].apply(len) < 10]
master_women[master_women.duplicated(subset=['Business Name','Email','Phone'])]

#Afterwards manually sift through the duplicated rows
master_women[master_women['Email'].duplicated()].sort_values(by=['Email'])

# Drop Null NA by specified column with NA values

In [ ]:
df = df.dropna(subset=['NIGP'])

In [ ]:
mvf = mvf[mvf['Fixed Ethnicity'].notnull()]

# Rename column

In [ ]:
df.rename(columns={'Old String':'New String'},inplace=True)

In [ ]:
#Fix for the sitatuion where merging causes duplicate rows
#I believe this is caused by a scenario where you are trying to merge with a table that
#Is not explicitly defined to be a string, so you have to explicitly define it as this
df1['Cost Center Code'] = df1['Cost Center Code'].astype(str)
df1['Cost Center Name'] = df1['Cost Center Name'].astype(str)
df2['Department Code'] = df2['Department Code'].astype(str)
df2['Department Name'] = df2['Department Name'].astype(str)
#Then do merge

# Iterate through dataframe and apply unique function to modify each value in a different way

In [ ]:
def mfd_zip(df):
    for index, row in df.iterrows():
        if(df.iloc[index]['Zip'][0].isalpha()):
            df.at[index,'MFD'] = 'Out of Country Zip'
        elif(df.iloc[index]['Zip'] == '00000'):
            df.at[index,'MFD'] = 'Invalid Zip Code'

# Assigning booleans based on a column, checking if string is a number

In [ ]:
df['Is NIGP'] = df['NIGP'].str.isdigit()

In [ ]:
Check if string is positive digit (integer) and alphabet
You may use str.isdigit() and str.isalpha() to check whether given string is positive integer and alphabet respectively.

Sample Results:

# For alphabet
>>> 'A'.isdigit()
False
>>> 'A'.isalpha()
True

# For digit
>>> '1'.isdigit()
True
>>> '1'.isalpha()
False

# Marking Data Gaps

In [ ]:
# Hard exclusions
#df['Award Date']=pd.to_datetime(df['Award Date'],infer_datetime_format=True) 
df.loc[(~(df['Award Date']>='1-1-14')|~(df['Award Date']<='12-31-18'))&(df['MFD']).isnull(),'MFD'] = 'Outside Date Range'
df.loc[((df['Award Amount'].isnull())|(df['Award Amount']==0))&(df['MFD'].isnull()),'MFD'] = 'Null Award Amount'
df.loc[(df['Award Amount']<1000)&(df['MFD']).isnull(),'MFD'] = 'Less than 1000'

# Creating a function to mark exclusions where multiple individual items can be placed in the MFD column The goal should be to preserve
# what is already in MFD and to exclude rows that have much deeper reasons for exclusions such as being exclusion categories or less than threshold map
def mark_gaps(df):
    # Set a boolean so that we can maintain the integrity of not overwriting those rows that have deeper exclusion reasons 
    # (hence the benefit of the MFD.isnull check in the loc statements)
    df.loc[df['MFD'].notnull(), 'Hard Exclusion'] = 'Y'
    # Make all null values empty strings
    df['MFD'].replace(np.nan, '', inplace=True)
    # Marking Gaps
    df.loc[(df['Zip'].isnull()) &
           (df['Hard Exclusion'].isnull()), 'MFD'] = df['MFD'] + 'No Zip,'
    df.loc[(df['Work Category'].isnull()) &
           (df['Hard Exclusion'].isnull()), 'MFD'] = df['MFD'] + 'No WC,'
    df.loc[(df['Work Category'] == '?') &
           (df['Hard Exclusion'].isnull()), 'MFD'] = df['MFD'] + 'No WC,'
    # Fixing the good rows so they have null values in the MFD again
    df['MFD'].replace('',np.nan ,inplace=True)
    return df

# Converting Force forcing multiple rows to a single row and preserving unique data condense consolidate

In [ ]:
def force_single_row(df,force_column,primary_key):
    df[force_column] = df[force_column].astype(str)
    joined_phones=df.groupby(primary_key)[force_column].apply(';'.join).reset_index()
    joined_phones.rename(columns={force_column:'All '+force_column+'s'},inplace=True)
    df = df.merge(joined_phones,on=primary_key,how='left')
    df.drop_duplicates(subset=[primary_key],inplace=True)
    return df

email_master = force_single_row(email_master,'Email','VENDOR_NBR')

In [ ]:
construction['Phone'] = construction['Phone'].astype(str)
joined_phones=construction.groupby('Company Name')['Phone'].apply(';'.join).reset_index()
joined_phones.rename(columns={'Phone':'All Phones'},inplace=True)
construction = construction.merge(joined_phones,on='Company Name',how='left')
construction.drop_duplicates(subset='Company Name',inplace=True)

# Filtering so only rows with true in column remain

In [ ]:
df = df[df['Is NIGP']]

# Convert NAICS to NIGP

In [ ]:
def naics_to_nigp(master,path):
    crosswalk = pd.read_excel(path)
    crosswalk['NAICS'] = crosswalk['NAICS'].astype(str)
    crosswalk['NIGP 5'] = crosswalk['NIGP_CODE'].astype(str)
    crosswalk['NIGP 3'] = crosswalk['NIGP 5'].astype(str).apply(lambda x: x[:3])
    crosswalk = crosswalk[['NAICS','NIGP 5','NIGP 3','NIGP_Description']]
    #There are multiple NIGP codes for a single NAICS code but I have already verified that it is arbitrary.
    crosswalk.drop_duplicates(subset=['NAICS'],inplace=True)

    def fix_nigp_5(string):
        string = str(string)
        if(len(string)==4):
            return '0'+string
        elif(len(string)==3):
            return '00'+string
        else:
            return string
    
    crosswalk['NIGP 5'] = crosswalk['NIGP 5'].apply(fix_nigp_5)
    master = master.merge(crosswalk,on='NAICS',how='left')
    return master

master = naics_to_nigp(master,'../NAICS-CROSSWALK.xlsx')

# Convert NIGP to Work Category

In [ ]:
# This function requires the master frame to have an NIGP 3 and NIGP 5 column
def nigp_to_work_category(master,path):
    nigp = pd.read_excel(path)
    # Making decisions as to how to classify certain codes
    nigp.loc[nigp['Work Category']=='Non-Commercial Activity','Work Category'] = 'Goods & Supplies'
    nigp.loc[nigp['Work Category']=='Non-Commerical Transactions','Work Category'] = 'Other Services'
    nigp.loc[nigp['Work Category']=='Non-Professional Services','Work Category'] = 'Other Services'
    
    # Fixing NIGP Codes so that they can merge
    def fix_nigp_3(string):
        string=str(string)
        if(len(string)==1):
            return '00'+string
        elif(len(string)==2):
            return '0'+string
        else:
            return string
    
    def fix_nigp_5(string):
        string = str(string)
        if(len(string)==4):
            return '0'+string
        elif(len(string)==3):
            return '00'+string
        elif(len(string)==2):
            return '000'+string
        else:
            return string
    
    nigp['NIGP 3'] = nigp['NIGP 3'].apply(fix_nigp_3)
    nigp['NIGP 5'] = nigp['NIGP 5'].apply(fix_nigp_5)
    
    nigp_3 = nigp[['NIGP 3','Commodity Description','Work Category']].copy()
    nigp_5 = nigp[['NIGP 5','Commodity Description','Work Category']].copy()
    
    nigp_3.rename(columns={'Commodity Description':'Commodity Description 3','Work Category':'Work Category 3'},inplace=True)
    nigp_5.rename(columns={'Commodity Description':'Commodity Description 5','Work Category':'Work Category 5'},inplace=True)
    
    master = master.merge(nigp_3,on='NIGP 3',how='left')
    master = master.merge(nigp_5,on='NIGP 5',how='left')
    
    # Filling work final work category in order of most significant matches to least
    master['Final Work Category'] = master['Work Category 5']
    master['Final Work Category'].fillna(master['Work Category 3'], inplace=True)
    
    return master

master = nigp_to_work_category(master,'../NIGP to Work Category from Rom - Cleaned - 190725ob.xlsx')

# Relevant Market

In [ ]:
#Creating Unique Arrays for the Relevant Market Regions
Main_County = {
    'Hamilton County':'TN'
}
MSA_Counties = {
    'Catoosa County':'GA',
    'Dade County':'GA',
    'Marion County':'TN',
    'Sequatchie County':'TN',
    'Walker County':'GA'
}
CSA_Counties = {
    'Bradley County':'TN',
    'Jackson County':'AL',
    'Mcminn County':'TN',
    'Murray County':'GA',
    'Polk County':'TN',
    'Rhea County':'TN',
    'Whitfield County':'GA'
}
Surrounding_Counties = {
    'Bledsoe Couny':'TN',
    'Meigs County':'TN'
}
Main_State = 'TN'
Relevant_States = ['TN','GA','KY','VA','NC','AL','MS','MO','AR']
Relevant_Market_Order = ['Main County','MSA','CSA','Surrounding Counties','TN','GA','KY','VA','NC','AL','MS','MO','AR', 'USA']

In [ ]:
#This function checks if the value of the passed dictionary matches the passed state
#For this to work, the dictionary needs to be inverted because searching a dictionary
#only works on keys, so the key value pair needs to be flipped

#This is a helper function for the create_relevant_market function

def check_inverse_mapping(state,dictionary):
    #Inverting the mapping of the dictionary so I can check both key and value in a one line if statement
    #https://stackoverflow.com/questions/483666/python-reverse-invert-a-mapping
    inv_map = {}
    for k, v in dictionary.items():
        #inv_msa[v] = inv_msa.get(v, [])
        #inv_msa[v].append(k)
        inv_map.setdefault(v, []).append(k)
        
    if state in inv_map:
        return True
    else:
        return False

In [ ]:
def create_relevant_market(df):
    df['Relevant Market Region'] = ''

    for index, row in df.iterrows():
        temp_state = df.iloc[index]['State'].strip()
        temp_county = df.iloc[index]['County'].strip()
        if temp_county in Main_County and temp_state == Main_County[temp_county]:
            df.at[index,'Relevant Market Region'] = 'Main County'
        elif temp_county in MSA_Counties and check_inverse_mapping(temp_state,MSA_Counties):
            df.at[index,'Relevant Market Region'] = 'MSA'
        elif temp_county in CSA_Counties and check_inverse_mapping(temp_state,CSA_Counties):
            df.at[index,'Relevant Market Region'] = 'CSA'
        elif temp_county in Surrounding_Counties and check_inverse_mapping(temp_state,Surrounding_Counties):
            df.at[index,'Relevant Market Region'] = 'Surrounding Counties'
        elif temp_state in Relevant_States:
            df.at[index,'Relevant Market Region'] = temp_state
        else:
            #Else it is within the USA
            df.at[index,'Relevant Market Region'] = 'USA' 

# Fix Clean NIGP 

In [ ]:
def fix_nigp_3(string):
    string=str(string)
    if(len(string)==1):
        return '00'+string
    elif(len(string)==2):
        return '0'+string
    else:
        return string
    
    rom_nigp['NIGP 3'] = rom_nigp['NIGP 3'].apply(fix_nigp_3)

In [ ]:
def fix_nigp_5(string):
    string = str(string)
    if(len(string)==4):
        return '000'+string
    elif(len(string)==3):
        return '00'+string
    else:
        return string
    
rom_nigp['NIGP 5'] = rom_nigp['NIGP 5'].apply(fix_nigp_5)

In [ ]:
def fix_nigp_7(string):
    string = str(string)
    if(len(string)==6):
        return '0'+string
    elif(len(string)==5):
        return '00'+string
    elif(len(string)==4):
        return '000'+string
    elif(len(string)==3):
        return '0000'+string
    elif(len(string)==2):
        return '00000'+string
    else:
        return string
    
seven_description['NIGP'] = seven_description['NIGP'].apply(fix_nigp_7)

# Applying Functions

In [ ]:
def times2(x):
    return x*2


In [ ]:
df['col1'].apply(times2)

In [ ]:
df['col3'].apply(len)

In [ ]:
df['col1'].sum()

# Copy a dataframe to avoid modifying a slice

In [ ]:
#https://stackoverflow.com/questions/38835483/confusion-re-pandas-copy-of-slice-of-dataframe-warning
chat = chat[['Source','Name','Owner','DBE Category','Office Phone','Final Ethnicity']].copy()

# Distance between two strings Levenshtein Distance similarity

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Number of Instances

In [ ]:
def getNumberOfInstances(df,name_column):
    name_counts = df[name_column].value_counts().rename('Number of Instances')

    dup_names = df.merge(name_counts.to_frame(),
                                    left_on=name_column,
                                    right_index=True)
    return dup_names

df = getNumberOfInstances(df,'VENDOR_NBR')


# Clean Simple Cleaning Phone Numbers Email

In [ ]:
def clean_phone(string):
    string = str(string).replace(" ", "").replace("(","").replace(")","").replace("-","").replace(".","")
    string = str(string)[:10]
    
    if(len(string)<10):
        return np.nan
    else:
        return string

master['Phone'] = master['Phone'].apply(clean_phone)

In [ ]:
def clean_zip(string):
    string = str(string)[:5]
    
    if(len(string)<5):
        return np.NaN
    else:
        return string

master['Zip'] = master['Zip'].apply(clean_zip)

In [ ]:
def clean_email(string):
    string = str(string).lower().strip()
    
    if(len(string)<5):
        return np.NaN
    else:
        return string

master['Email'] = master['Email'].apply(clean_email)

# Lambda conditional if else

In [ ]:
df['c']=df['b'].apply(lambda x: 0 if x ==0 else math.log(x))

# Correcting Zip Codes Syntax Print Line Parser

In [ ]:
po[po['County'].isnull()]['Zip Code'].unique()

incorrect_zips = ['02129', '07647', '08066', 'R3B 0', '08107', '01730', '07960',
       '06450', '01742', '06904', 'V8W 1', '08054', '02451', '02142',
       '01441', '06820', '07728', '06461', '02809', '06810', '01040',
       '02241', '44204', '06484', '03820', '01966', '02026', '05101',
       '02109', '02038', '01373', '06010', '22767', '06492', '02145',
       '02446', '01752', '02494', '07801', '06475', '08085', 'K1G 6',
       '08701', '02072', '08110', '02116', '07083', '08361', '07511',
       '01719', 'EH2 2', '02210', 'E8 4R', '08057', '08033']

def zip_code_syntax_corrector(array):
    for i in range(len(array)):
        print('po.loc[po[\'Zip Code\']==\''  +str(array[i])+'\',\'County\'] = \'\'')
        
        
zip_code_syntax_corrector(incorrect_zips)

# Clean Cleaning Names

In [ ]:
def cleanest_names(df,name_column,no_new_column):
    df['Cleanest Name'] = df[name_column]
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x).replace("&",''))
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x).replace("-",''))
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x).replace(',',''))
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x).replace('.',''))
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x).replace("'",''))
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x).upper())
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x).lstrip())
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x).rstrip())
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x)[:-4] if x.endswith('llc') else x)
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x)[:-5] if x.endswith('pllc') else x)
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x)[:-4] if x.endswith('inc') else x)
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x)[:-4] if x.endswith('ltd') else x)
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x)[:-5] if x.endswith('corp') else x)
    df['Cleanest Name'] = df['Cleanest Name'].apply(lambda x: str(x).replace(" ",''))
    
    if(no_new_column):
        df[name_column] = df['Cleanest Name']
        df.drop(['Cleanest Name'], axis=1,inplace=True) 

In [ ]:
def clean_names(df,name_column,no_new_column):
    df['Cleaned Name'] = df[name_column]
    df['Cleaned Name'] = df['Cleaned Name'].apply(lambda x: str(x).replace(',',''))
    df['Cleaned Name'] = df['Cleaned Name'].apply(lambda x: str(x).replace('.',''))
    df['Cleaned Name'] = df['Cleaned Name'].apply(lambda x: str(x).replace("'",''))
    df['Cleaned Name'] = df['Cleaned Name'].apply(lambda x: str(x).title())
    df['Cleaned Name'] = df['Cleaned Name'].apply(lambda x: str(x).lstrip())
    df['Cleaned Name'] = df['Cleaned Name'].apply(lambda x: str(x).rstrip())
    df['Cleaned Name'] = df['Cleaned Name'].apply(lambda x: str(x)[:-4] if x.endswith('Llc') else x)
    df['Cleaned Name'] = df['Cleaned Name'].apply(lambda x: str(x)[:-5] if x.endswith('Pllc') else x)
    df['Cleaned Name'] = df['Cleaned Name'].apply(lambda x: str(x)[:-4] if x.endswith('Inc') else x)
    df['Cleaned Name'] = df['Cleaned Name'].apply(lambda x: str(x)[:-4] if x.endswith('Ltd') else x)
    df['Cleaned Name'] = df['Cleaned Name'].apply(lambda x: str(x)[:-5] if x.endswith('Corp') else x)
    
    if(no_new_column):
        df[name_column] = df['Cleaned Name']
        df.drop(['Cleaned Name'], axis=1,inplace=True) 

# Award File Columns needed

In [ ]:
# Awards + Vendors
df = df[['Number of Instances', 'MFD', 'Source', 'Pulled From', 'ACTIVE_FLAG','Business Name',
       'Smoothed Name', 'Owner', 'Phone', 'Email', 'Address',
       'City', 'County', 'State', 'Zip', 'DBE Category', 'Work Description',
       'NIGP 5', 'NIGP 3','Work Category',
       'Work Categorization Type', 'Potential Minority Identifier',
       'Relevant Market Region',
        'Po Number', 'Po Creation Date',
       'Status',
        'Po Item Description', 
       'Po Comments', 'Item Category',
       'Small Business Flag', 'Women Owned Flag', 
       'Actual Shipment Amount', 'Final Work Category',
       'Potentially Exclude',
       'Null Shipment Amount', 
       'Department Name', 'Ethnicity', 'Certification']].copy()

# AWARDS
po = po[['Source', 'MFD', 'Po Number', 'Po Creation Date',
       'Status', 'Po Supplier Name',
       'Zip', 'Po Item Description', 
       'Po Comments', 'Item Category',
       'Small Business Flag', 'Women Owned Flag', 
       'NIGP 7', 'NIGP 5', 'NIGP 3', 'Commodity Description 3',
       'Work Category 3', 'Commodity Description 5', 'Work Category 5',
       '7 Digit Lookup', 'Actual Shipment Amount', 'Final Work Category',
        'City', 'County', 'State', 'Potentially Exclude',
       'Null Shipment Amount', 
       'Department Name', 'Ethnicity', 'Certification', 'Relevant Market Region']]


# VENDORS
master = master[['Number of Instances','ACTIVE_FLAG', 'MFD', 'Source', 'Pulled From', 'Business Name',
       'Smoothed Name', 'VENDOR_NBR', 'Owner', 'Phone', 'Email', 'Address',
       'City', 'County', 'State', 'Zip', 'DBE Category', 'Work Description',
       'NIGP 5', 'NIGP 3', 'Ethnicity', 'Certification', 'Work Category',
       'Work Categorization Type', 'Potential Minority Identifier',
       'Relevant Market Region', 'Potentially Exclude']].copy()

# Expand Explode semi colon separated values into new rows

In [ ]:
def explode_naics(df):
    s = df["Extracted NAICS"].str.split(',', expand=True).stack()
    i = s.index.get_level_values(0)
    df2 = df.loc[i].copy()
    df2["New NAICS"] = s.values
    
    #Strip is important here because it removes white spaces
    df2['New NAICS'] = df2['New NAICS'].apply(lambda x: x.strip())
    return df2

In [ ]:
def explode_email(df):
    s = df["Email Address"].str.split(';', expand=True).stack()
    i = s.index.get_level_values(0)
    df2 = df.loc[i].copy()
    df2["Email Address"] = s.values
    return df2

# Create horizontal bar create meta table

In [ ]:
def createMetaTable(df):
    col_names = []
    col_amount = []
    col_unique = []

    for col in df.columns: 
        col_names.append(col) 
        
        col_name = str(col)
        temp = df.dropna(subset=[col_name])
        col_amount.append(len(temp[col_name]))
        
        col_unique.append(len(temp[col_name].unique()))

    # intialise data of lists. 
    data = {'Column Name':col_names, 'Fill Amount':col_amount,'Number Unique':col_unique} 

    # Create DataFrame 
    df = pd.DataFrame(data)
    df.sort_values(by=['Fill Amount','Number Unique'],ascending=False,inplace=True)
    
    # Marking columns with less than 5 percent of the max number of rows
    maxValue = df['Fill Amount'].max()
    df.loc[df['Fill Amount']<(0.05*maxValue),'Less than 5 Percent'] = 'Y'
    
    return df

def createLayeredBar(meta,path):
    import altair as alt
    
    # Ref: https://altair-viz.github.io/user_guide/generated/core/altair.EncodingSortField.html
    # Ref: https://altair-viz.github.io/user_guide/encoding.html#sorting-legends
    # Ref: https://vega.github.io/vega/docs/schemes/
    
    meta_fill = meta[['Column Name','Fill Amount']].copy()
    meta_fill.rename(columns={'Fill Amount':'Amount'},inplace=True)
    meta_unique = meta[['Column Name','Number Unique']].copy()
    meta_unique.rename(columns={'Number Unique':'Amount'},inplace=True)
    meta_fill['Type'] = 'Fill Amount'
    meta_unique['Type'] = 'Number Unique'
    meta = pd.concat([meta_fill,meta_unique])

    alt.renderers.enable('notebook')

    chart = alt.Chart(meta).mark_bar(opacity=0.7).encode(
        x=alt.X('Amount:Q',stack=None),
        y=alt.Y('Column Name:O',sort=alt.EncodingSortField(field='Amount',op='count',order='ascending')),
        #color="Type",
        color=alt.Color('Type', scale=alt.Scale(scheme='dark2')),
        tooltip=['Column Name', 'Type', 'Amount']
    ).interactive()
    
    chart.save(path)
    
    return chart

In [2]:
%matplotlib inline 
import seaborn as sns
import matplotlib.pyplot as plt

def createMetaTable(df):
    col_names = []
    col_amount = []
    col_unique = []

    for col in df.columns: 
        col_names.append(col) 
        
        col_name = str(col)
        temp = df.dropna(subset=[col_name])
        col_amount.append(len(temp[col_name]))
        
        col_unique.append(len(temp[col_name].unique()))

    # intialise data of lists. 
    data = {'Column Name':col_names, 'Fill Amount':col_amount,'Number Unique':col_unique} 

    # Create DataFrame 
    df = pd.DataFrame(data)
    df.sort_values(by=['Fill Amount','Number Unique'],ascending=False,inplace=True)
    
    # Marking columns with less than 5 percent of the max number of rows
    maxValue = df['Fill Amount'].max()
    df.loc[df['Fill Amount']<(0.05*maxValue),'Less than 5 Percent'] = 'Y'
    
    return df

def createHorizontalBar(df,x_col,y_col,title,path,filter_gapless):
    maxValue = df[x_col].max()
    
    if(filter_gapless==True):
        df = df[df[x_col]!=maxValue].copy()
    else:
        pass
    
    df.sort_values(by=[x_col],ascending=False,inplace=True)
    f, ax = plt.subplots(figsize = (30,15))
    p = sns.barplot(data=df, y = y_col,x=x_col).set_title(title)
    
    
    
    #looping through every bar
    for p in ax.patches:
        _x = p.get_x() + p.get_width() + 0.4
        _y = p.get_y() + p.get_height()
        # Storing the text to be output as data label in 'value', p.get_width() retrieves the value
        value = ' '+ str(int(p.get_width()))+' / '+str(round((int(p.get_width())/maxValue)*100,2))+'%'
        ax.text(_x, _y, value, ha="left")
                    
    p.figure.savefig(path)
    
# meta = createMetaTable(df)
# createHorizontalBar(meta,'Fill Amount','Column Name','PRO Data Gaps','exports/191002/PRO Data Gaps.png',False)
# createHorizontalBar(meta,'Number Unique','Column Name','PRO Unique Count','exports/191002/PRO Unique Count.png',False)

NameError: name 'df' is not defined

# Convert date to datetime

In [ ]:
master['Award Date']=pd.to_datetime(master['Award Date'],infer_datetime_format=True) 

# Pandas Profile Profiling

In [ ]:
import pandas_profiling as pp
# df = pd.read_excel('DESKTOP/EIFNAISDAF.XLSX')
profile = pp.ProfileReport(df)
profile.to_file(outputfile="../references/profiles/Profile_.html")

# JSON Output
# pp.describe_df(DF_NAME)

# Count number of occurances of a character

In [ ]:
df_new['Email Count'] = df_new['Email Address'].str.count('@')

# Get Mean Median

In [ ]:
construction.describe()

# Filter Null amounts

In [ ]:
def filter_null_shipment(x):
    if(x==0):
        return True
    elif(x!=x):
        return False
    else: 
        reurn False

In [ ]:
po['Null Shipment Amount'] = po['Actual Shipment Amount'].apply(filter_null_shipment)

# Dataframe Value Count for Excel Graphing Bar Info

In [ ]:
for col in data.columns: 
    print(col) 

In [ ]:
for col in po.columns:
    col_name = str(col)
    temp = po.dropna(subset=[col_name])
    print(len(temp[col_name]))

# Make lower case upper case

In [ ]:
master_2['Email'] = master_2['Email'].str.lower

# Zip Code Merge

In [ ]:
zipcodes = pd.read_excel('../zip_code_database-171005.xls')

zipcodes.rename(columns={'zip':'Zip','county':'County','state':'State Merged'},inplace=True)
zipcodes['Zip'] = zipcodes['Zip'].astype(str)
zipcodes=zipcodes[['Zip','County','State Merged']].copy()

def clean_zip(string):
    string = str(string)[:5]
    
    if(len(string)==2):
        return '000'+string
    elif(len(string)==3):
        return '00'+string
    elif(len(string)==4):
        return '0'+string
    else:
        return string
    
df['Zip'] = df['Zip'].apply(clean_zip)
zipcodes['Zip'] = zipcodes['Zip'].apply(clean_zip)
df = df.merge(zipcodes,on='Zip',how='left')
df['State'] = df['State Merged']
df[df['County'].isnull()]['Zip'].value_counts()
#df.loc[df['County'].isnull(),'MFD'] = 'Invalid Zip'

In [ ]:
zipcodes = pd.read_excel('../zip_code_database-171005.xls')
zipcodes.rename(columns={'zip':'Zip','county':'County'},inplace=True)
zipcodes['Zip'] = zipcodes['Zip'].astype(str)
zipcodes=zipcodes[['Zip','County']].copy()
def clean_zip(string):
    string = str(string)[:5]
    
    if(len(string)<5):
        return np.nan
    else:
        return string

vendors['Zip'] = vendors['Zip'].apply(clean_zip)
vendors = vendors.merge(zipcodes,on='Zip',how='left')

# Assigning values to a column based on values of another column

In [ ]:
po.loc[po['Null Shipment Amount']==True,'MFD'] = 'Null or Zero Shipment Amount'

# Merging on shared column updating old values with new values

In [ ]:
esther_work = esther_work[['Business Name','New Ethnicity']]
master = master.merge(esther_work, on='Business Name',how='left')
#This assigns any new values that were brought in to overwrite the old values
master.loc[master['New Ethnicity'].notnull(),'Ethnicity'] = master['New Ethnicity']
master = master.drop(labels=['New Ethnicity'], axis=1)

# Changing default number of rows columns displayed

In [ ]:
pd.set_option('display.max_rows',2000)

In [ ]:
pd.set_option('display.max_columns',None)

# Convert pdf to dataframe excel csv

In [ ]:
import pandas as pd
import tabula

df = read_pdf('data/asdfsadf.pdf')
tabula.convert_into('inout path',output_format="csv")

# Format Phone Numbers

In [ ]:
def phone_format(n):                                                                                                                                  
    if(n[0].isdigit()==False):
        return n
    else:
        return format(int(n[:-1]), ",").replace(",", "-") + n[-1] 

# Drop Duplicates

In [ ]:
po.drop_duplicates(subset='Po Supplier Name',inplace=True)

# Drop Columns

In [ ]:
df.drop(['B', 'C'], axis=1)

# Pivot Table

In [ ]:
po_pivot = pd.pivot_table(po,index=['Po Supplier Name'], values='ID', aggfunc=pd.Series.nunique)

# Flattening the pivot table so it is usable as a dataframe

In [ ]:
construction_pivot = pd.DataFrame(construction_pivot.to_records())

# Sort

In [ ]:
po_pivot.sort_values(by=['ID'],ascending=False)

# Create MFD Columns

In [ ]:
import numpy as np

def mfd_zip(df):
    for index, row in df.iterrows():
        if(df.iloc[index]['Zip'][0].isalpha()):
            df.at[index,'MFD'] = 'Out of Country Zip'
        elif(df.iloc[index]['Zip'] == '00000'):
            df.at[index,'MFD'] = 'Invalid Zip Code'
        elif(df.iloc[index]['Zip'] == '00nan'):
            df.at[index,'MFD'] = 'Invalid Zip Code'
        elif(df.iloc[index]['Zip'] == '00820'):
            df.at[index,'County'] = 'Saint Croix'
        elif(df.iloc[index]['Zip'] == '07623'):
            df.at[index,'Zip'] = '07632'
            df.at[index,'City'] = 'Englewood Cliffs'
            df.at[index,'County'] = 'Bergen County'
            df.at[index,'State'] = 'NJ'
        elif(df.iloc[index]['Zip'] == '20802'):
            df.at[index,'Zip'] = '80202'
            df.at[index,'City'] = 'Denver'
            df.at[index,'County'] = 'Denver County'
            df.at[index,'State'] = 'CO'
        elif(df.iloc[index]['Zip'] == '30057'):
            df.at[index,'MFD'] = 'Invalid Zip Code'
        elif(df.iloc[index]['Zip'] == '37418'):
            df.at[index,'Zip'] = '37408'
            df.at[index,'City'] = 'Chattanooga'
            df.at[index,'County'] = 'Hamilton County'
            df.at[index,'State'] = 'TN'
        elif(df.iloc[index]['Zip'] == '37440'):
            df.at[index,'Zip'] = '37410'
            df.at[index,'City'] = 'Chattanooga'
            df.at[index,'County'] = 'Hamilton County'
            df.at[index,'State'] = 'TN'
        elif(df.iloc[index]['Zip'] == '37446'):
            df.at[index,'Zip'] = '37416'
            df.at[index,'City'] = 'Chattanooga'
            df.at[index,'County'] = 'Hamilton County'
            df.at[index,'State'] = 'TN'
        elif(df.iloc[index]['Zip'] == '37463'):
            df.at[index,'Zip'] = '37403'
            df.at[index,'City'] = 'Chattanooga'
            df.at[index,'County'] = 'Hamilton County'
            df.at[index,'State'] = 'TN'
        elif(df.iloc[index]['Zip'] == '37471'):
            df.at[index,'Zip'] = '37421'
            df.at[index,'City'] = 'Chattanooga'
            df.at[index,'County'] = 'Hamilton County'
            df.at[index,'State'] = 'TN'
        elif(df.iloc[index]['Zip'] == '38309'):
            df.at[index,'Zip'] = '37309'
            df.at[index,'City'] = 'Calhoun'
            df.at[index,'County'] = 'McMinn County'
            df.at[index,'State'] = 'TN'
        elif(df.iloc[index]['Zip'] == '41247'):
            df.at[index,'MFD'] = 'Invalid Zip Code'
        elif(df.iloc[index]['Zip'] == '49536'):
            df.at[index,'MFD'] = 'Out of Country Zip'
        elif(df.iloc[index]['Zip'] == '60904'):
            df.at[index,'Zip'] = '30904'
            df.at[index,'City'] = 'Augusta'
            df.at[index,'County'] = 'Richmond County'
            df.at[index,'State'] = 'GA'
        elif(df.iloc[index]['Zip'] == '70100'):
            df.at[index,'MFD'] = 'Out of Country Zip'   
        #else:
            #df.at[index,'MFD'] = np.nan

# Excel

# Delete Blank Rows

In [ ]:
https://www.excel-easy.com/examples/delete-blank-rows.html